In [ ]:
# # IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# # RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
# import kagglehub




# kagglehub.login()


In [ ]:
# # IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# # THEN FEEL FREE TO DELETE THIS CELL.
# # NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# # ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# # NOTEBOOK.

# ee_5179_modern_computer_vision_course_competition_path = kagglehub.competition_download('ee-5179-modern-computer-vision-course-competition')
# stylishkabutar_bsds500_path = kagglehub.dataset_download('stylishkabutar/bsds500')

# print('Data source import complete.')


In [ ]:

# ee_5179_modern_computer_vision_course_competition_path

In [ ]:
# import os
# file=('/kaggle/input')
# print(os.listdir(file))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms,models
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
# from scipy.io import loadmat
# from skimage import io
# import cv2

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("0jl/NYUv2", trust_remote_code=True, split="train")


In [ ]:
# import os
# from PIL import Image
# from torchvision import datasets

# # Assuming 'dataset' is your PyTorch dataset
#   # Replace with your actual dataset

# # Define the target directory
# target_directory = 'nyu2'
# gt_directory='nyu2l'
# os.makedirs(target_directory, exist_ok=True)

# # Loop through the dataset and save images
# for idx, outputs in enumerate(dataset):
#     image_path = os.path.join(target_directory, f'image_{idx}.png')
#     outputs['image'].save(image_path)
#     # image_path = os.path.join(gt_directory, f'image_{idx}.png')
#     # outputs['depth'].save(image_path)# Save the image using PIL

# print("Images have been successfully saved to the target folder!")


In [ ]:
# max((dataset[0]['depth']))


In [ ]:
# ### Code to create synthetic Binary Images from RGB Images ####
# ### Just input the folder in BASE and it will output the binary images in BASE__1 ###
# import numpy as np
# import os, sys
# import cv2
# # Original Clean Images
# BASE = 'nyu2'
# frames = [1]

# TOTAL_FRAMES=frames[-1]
# imgs = sorted(os.listdir(BASE))
# for idx in range(len(imgs)):
#     im = cv2.imread(os.path.join(BASE, imgs[idx]))
#     im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
#     count_shape = im.shape + (-1,)

#     for i in range(1):
#         # Simulate low light conditions by scaling pixel values by 1/1000.
#         # Poisson process to simulate photon arrival
#         photon_counts = np.random.poisson(im.flatten()/255., size=(TOTAL_FRAMES, im.size)).T
#         photon_counts = photon_counts.reshape(count_shape)
#         # Photon counts is converted to binary frames

#         b_counts=np.where(photon_counts>=1, 1, 0)
#         for fil in frames:
#             recon_image = np.mean(b_counts[:,:,0:fil], axis=2)
#             #outfile = BASENEW.replace('60', str(fil)) + str(i) + im_name.replace('JPEG', 'png')
#             outfile = os.path.join(BASE+"__"+str(fil), imgs[idx])
#             directory = os.path.dirname(outfile)
#             if not os.path.exists(directory):
#                 os.makedirs(directory)

#             # recon_image = np.repeat(np.expand_dims(recon_image, axis = -1), 3, axis = -1)
#             recon_image = np.expand_dims(recon_image, axis = -1)
#             cv2.imwrite(outfile, recon_image*255.)

In [1]:
# plt.imshow(plt.imread('/kaggle/working/nyu2__1/image_0.png'))

In [2]:
# from IPython.display import FileLink
# FileLink("predictions_hed_resss_cub7.csv")

In [3]:
# import torch
# from torchvision import transforms
# from torchvision.datasets import ImageFolder
# from torch.utils.data import DataLoader
# from PIL import Image
# import numpy as np

# # Define multiscale Gaussian filters
# scales = [1, 2, 3, 5]  # Different sigma values
# filters = [transforms.GaussianBlur(kernel_size=(2*s+1, 2*s+1), sigma=s) for s in scales]

# # Custom transform to apply Gaussian filters and stack as channels
# class MultiscaleGaussianTransform:
#     def __init__(self, filters):
#         self.filters = filters

#     def __call__(self, img):
#         # Apply each filter and stack results as channels
#         filtered_images = [f(img) for f in self.filters]
#         filtered_images.append(img)
#         stacked_image = torch.stack([transforms.ToTensor()(fi).squeeze(0) for fi in filtered_images], dim=0)
#         return stacked_image

# # Dataset and DataLoader
# dataset_path = "path/to/your/dataset"  # Replace with your dataset path

# transform = MultiscaleGaussianTransform(filters)  # Apply your custom multiscale transform

# # dataset = ImageFolder(root=dataset_path, transform=transform)
# # dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# # # Process the dataset
# # for batch_idx, (images, labels) in enumerate(dataloader):
# #     print(f"Batch {batch_idx}: {images.shape}")  # Shape: [batch_size, num_filters, C, H, W]


In [4]:
# import os

# # Create the directory
# os.makedirs('/kaggle/working/test_fold', exist_ok=True)


Here we see that the canny detector does extract very sharp edges, but it either msises edges, or extracts too many, as it extracts edges at a particular scale

In [5]:
class BSDS500Dataset(Dataset):
    def __init__(self, image_dir, gt_dir, train_transform=None,test_transform=None):
        self.image_paths =os.listdir(image_dir)


        self.image_dir=image_dir
        self.gt_dir=gt_dir
        self.test_transform = test_transform
        self.train_transform=train_transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load image
        image = Image.open(self.image_dir+'/'+self.image_paths[idx])
        gt = Image.open(self.gt_dir+'/'+self.image_paths[idx])
        # Load ground truth

        if image.size[0]!=gt.size[0]:
            gt = gt.transpose(Image.ROTATE_90)
        if image.size[0]>image.size[1]:
            image = image.transpose(Image.ROTATE_90)
            gt = gt.transpose(Image.ROTATE_90)

        # Apply transformations
        if self.train_transform:
            image = self.train_transform(image)
        if self.test_transform:
            gt = self.test_transform(gt)

        return image, gt

NameError: name 'Dataset' is not defined

In [ ]:
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev


In [ ]:
!pip install torch-xla
!pip install cloud-tpu-client


In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

# Send model to TPU device
device = xm.xla_device()
# model = model.to(device)


In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
base_dir ='/kaggle/input/ee-5179-modern-computer-vision-course-competition/competition-data'
train_img_dir = os.path.join('/kaggle/input/ee-5179-modern-computer-vision-course-competition/competition-data/training-images')
train_gt_dir = os.path.join('/kaggle/input/ee-5179-modern-computer-vision-course-competition/competition-data/training-depths')
val_img_dir = os.path.join('/kaggle/input/ee-5179-modern-computer-vision-course-competition/competition-data/validation-images')
val_gt_dir = os.path.join('/kaggle/input/ee-5179-modern-computer-vision-course-competition/competition-data/validation-depths')
test_img_dir = os.path.join('/kaggle/input/ee-5179-modern-computer-vision-course-competition/testing-images')
transform = transforms.ToTensor()

transform1 = transforms.ToTensor()
train_dataset = BSDS500Dataset(train_img_dir, train_gt_dir, train_transform=transform,test_transform=transform1)
val_dataset = BSDS500Dataset(val_img_dir, val_gt_dir, train_transform=transform,test_transform=transform1)
train_sampler = torch.utils.data.distributed.DistributedSampler(
    train_dataset, num_replicas=xm.xrt_world_size(), rank=xm.get_ordinal()
)
val_sampler = torch.utils.data.distributed.DistributedSampler(
    val_dataset, num_replicas=xm.xrt_world_size(), rank=xm.get_ordinal()
)
# test_dataset = BSDS500Dataset(test_img_dir, test_gt_dir, transform=transform)
train_loader = DataLoader(train_dataset, sampler=train_sampler)
val_loader = DataLoader(val_dataset,sampler=val_sampler)
# test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:
# class NYUDEPTHV2(Dataset):
#     def __init__(self, image_dir, gt_dataset, train_transform=None,test_transform=None):
#         self.image_paths =os.listdir(image_dir)


#         self.image_dir=image_dir
#         self.gt_dataset=gt_dataset
#         self.test_transform = test_transform
#         self.train_transform=train_transform

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         # Load image
#         image = Image.open(self.image_dir+'/'+f'image_{idx}.png')

#         # Load ground truth
#         depth_array = np.array(self.gt_dataset[idx]['depth'])

# # Normalize the depth values to fit within the range of 0-255
#         depth_normalized = (depth_array - depth_array.min()) / (depth_array.max() - depth_array.min()) * 255
#         depth_normalized = depth_normalized.astype(np.uint8)

# # Convert the normalized depth array to a PIL image
#         gt = Image.fromarray(depth_normalized)
#         if image.size[0]!=gt.size[0]:
#             gt = gt.transpose(Image.ROTATE_90)
#         if image.size[0]>image.size[1]:
#             image = image.transpose(Image.ROTATE_90)
#             gt = gt.transpose(Image.ROTATE_90)
#         image = image.resize((481, 321))
#         gt = gt.resize((481, 321))
#         # Apply transformations
#         if self.train_transform:
#             image = self.train_transform(image)
#         if self.test_transform:
#             gt = self.test_transform(gt)

#         return image, gt

In [ ]:
# train_dataset_2=NYUDEPTHV2('/kaggle/working/nyu2__1',dataset, train_transform=transform,test_transform=transform1)

In [ ]:
# train_loader_2 = DataLoader(train_dataset_2, batch_size=4, shuffle=True)


In [ ]:
# plt.imshow(train_dataset_2[0][1][0])

In [ ]:
# models.resnet34(pretrained=False)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class HED(nn.Module):
    def __init__(self):
        super(HED, self).__init__()

        # Load pre-trained VGG16 features
        features = models.resnet34(pretrained=False)

        # Split VGG16 into blocks (until each pooling layer)
        self.conv0 = nn.Conv2d(1,3, kernel_size=1)
        self.conv1 = features.conv1
        self.bn1 = features.bn1
        self.rel = features.relu
        self.mp = features.maxpool
        self.conv11 = features.layer1  # until pool1
        self.conv2 = features.layer2 # until pool2
        self.conv3 = features.layer3 # until pool3
        self.conv4 = features.layer4 # until pool4
        self.conv5=features
        # self.conv5 = features[23:30] # until pool5 (exclude pool5)
        self.linear=nn.Linear(1000,1024)
        # Side output layers
        self.side1 = nn.Conv2d(64, 1, kernel_size=1)
        self.side2 = nn.Conv2d(256, 1, kernel_size=1)
        self.side3 = nn.Conv2d(256, 1, kernel_size=1)
        self.side4 = nn.Conv2d(512, 1, kernel_size=1)

        # self.side5 = nn.Conv2d(512, 1, kernel_size=1)

        # Fusion layer
        self.fusion = nn.Conv2d(5, 1, kernel_size=1)
        self.decoder4= nn.Sequential(
            nn.ConvTranspose2d(in_channels= 512,out_channels=256, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=256, out_channels=32, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=32, out_channels=8, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=8, out_channels=2, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
        nn.ConvTranspose2d(in_channels=2, out_channels=1, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True))
        self.decoder3= nn.Sequential(
            nn.ConvTranspose2d(in_channels=256, out_channels=16, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=16, out_channels=4, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=4, out_channels=2, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
        nn.ConvTranspose2d(in_channels=2, out_channels=1, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True))
        self.decoder2=nn.Sequential(

            nn.ConvTranspose2d(in_channels=256, out_channels=32, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=32, out_channels=8, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
        nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True))
        self.decoder1= nn.Sequential(

            nn.ConvTranspose2d(in_channels=64, out_channels=8, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
        nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True))
        self.decoder5= nn.Sequential(

            nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
        nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
        nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=4, stride=2, padding=1),  # Upscale x2
            nn.ReLU(inplace=True))
        # Initialize weights for side outputs and fusion
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        h, w = x.size(2), x.size(3)

        # VGG blocks forward pass
        x=self.conv0(x)
        side5=self.conv5(x)

        x=self.conv1(x)
        x=self.bn1(x)
        x=self.rel(x)
        x=self.mp(x)
        conv1 = self.conv11(x)
        y=x
        x=x+y.view(x.shape)
        y=x
        conv2 = self.conv2(conv1)
        x=x+y.view(x.shape)
        y=x
        conv3 = self.conv3(conv2)
        x=x+y.view(x.shape)
        y=x
        conv4 = self.conv4(conv3)
        x=x+y.view(x.shape)
        y=x
        # conv5 = self.conv5(conv4)

        # Side outputs
        side1 = self.decoder1(conv1)
        side2 = self.decoder2(conv2)
        side3 = self.decoder3(conv3)
        side4 = self.decoder4(conv4)
        side5=self.linear(side5)
        side5=side5.view((1,1,32,32))
        side5=self.decoder5(side5)

        # side5 = self.side5(conv5)

        # print(side1.shape,side2.shape,side3.shape,side4.shape)
        # Upsampling
        # side1 = F.interpolate(side1, size=(h, w), mode='bicubic', align_corners=True)
        # side2 = F.interpolate(side2, size=(h, w), mode='bicubic', align_corners=True)
        # side3 = F.interpolate(side3, size=(h, w), mode='bicubic', align_corners=True)
        # side4 = F.interpolate(side4, size=(h, w), mode='bicubic', align_corners=True)
        # batch_size, channels, height, width = side1.shape
        # flattened_tensor = side1.view(batch_size, -1)  # Flatten to 1D
        # fc_layer = nn.Linear(flattened_tensor.shape[1], flattened_tensor.shape[1])  # Input size = Output size
        # fc_output = fc_layer(flattened_tensor.to(device))
        # side1 = fc_output.view(batch_size, channels, height, width)
        # batch_size, channels, height, width = side2.shape
        # flattened_tensor = side2.view(batch_size, -1)  # Flatten to 1D
        # fc_layer = nn.Linear(flattened_tensor.shape[1], flattened_tensor.shape[1])  # Input size = Output size
        # fc_output = fc_layer(flattened_tensor.to(device))
        # side2 = fc_output.view(batch_size, channels, height, width)
        # batch_size, channels, height, width = side3.shape
        # flattened_tensor = side3.view(batch_size, -1)  # Flatten to 1D
        # fc_layer = nn.Linear(flattened_tensor.shape[1], flattened_tensor.shape[1])  # Input size = Output size
        # fc_output = fc_layer(flattened_tensor.to(device))
        # side3 = fc_output.view(batch_size, channels, height, width)
        # batch_size, channels, height, width = side4.shape
        # flattened_tensor = side1.view(batch_size, -1)  # Flatten to 1D
        # fc_layer = nn.Linear(flattened_tensor.shape[1], flattened_tensor.shape[1])  # Input size = Output size
        # fc_output = fc_layer(flattened_tensor.to(device))
        # side4 = fc_output.view(batch_size, channels, height, width)
        # side5 = F.interpolate(side5, size=(h, w), mode='bilinear', align_corners=True)

        # Fusion of side outputs
        # print(side1.shape,side2.shape,side3.shape,side4.shape,side5.shape)
        fused = torch.cat((side1, side2, side3, side4,side5), dim=1)
        fused = self.fusion(fused)

        # Apply sigmoid to all outputs
        # side1 = torch.sigmoid(side1)
        # side2 = torch.sigmoid(side2)
        # side3 = torch.sigmoid(side3)
        # side4 = torch.sigmoid(side4)
        # side5 = torch.sigmoid(side5)
        # fused = torch.sigmoid(fused)

        return side1, side2, side3, side4,side5, fused



# HED Loss
# def hed_loss(side_outputs, fused, labels, beta=1.1):
#     # Compute loss for each side output

#     loss1 = depth_smoothness_loss(side_outputs[0], labels)
#     loss2 =  depth_smoothness_loss(side_outputs[1], labels)
#     loss3 = depth_smoothness_loss(side_outputs[2], labels)
#     loss4 = depth_smoothness_loss(side_outputs[3], labels)
#     # loss5 =  depth_smoothness_loss(side_outputs[4], labels)

#     # Compute loss for fused output
#     loss_fused =depth_smoothness_loss(fused, labels)

#     # Total loss
#     loss_total = loss1 + loss2 + loss3 + loss4 +  loss_fused

#     return loss_total
def hed_loss(side_outputs, fused, labels, beta=1.1):
    # Compute loss for each side output
    MSELoss=nn.MSELoss()
    loss1 = 1 - ssim(side_outputs[0], labels, data_range=1, size_average=True)+MSELoss(side_outputs[0], labels)#+ depth_smoothness_loss(side_outputs[0], labels)
    loss2 =  1 - ssim(side_outputs[1], labels, data_range=1, size_average=True)+MSELoss(side_outputs[1], labels)#+ depth_smoothness_loss(side_outputs[1], labels)
    loss3 =  1- ssim(side_outputs[2], labels, data_range=1, size_average=True)+MSELoss(side_outputs[2], labels)#+ depth_smoothness_loss(side_outputs[2], labels)
    loss4 =  1- ssim(side_outputs[3], labels, data_range=1, size_average=True)+MSELoss(side_outputs[3], labels)#+ depth_smoothness_loss(side_outputs[3], labels)
    loss5 = 1 - ssim(side_outputs[4], labels, data_range=1, size_average=True)+MSELoss(side_outputs[4], labels)#+ depth_smoothness_loss(side_outputs[0], labels)

    # loss5 =  MSELoss(side_outputs[4], labels)

    # Compute loss for fused output
    loss_fused =1 - ssim(fused, labels, data_range=1, size_average=True)+MSELoss(fused, labels)#+depth_smoothness_loss(fused, labels)

    # Total loss
    loss_total = loss1 + loss2 + loss3 + loss4 + loss5+ loss_fused

    return loss_total

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class HED(nn.Module):
    def __init__(self):
        super(HED, self).__init__()

        # Load pre-trained VGG16 features
        features = models.vgg16(pretrained=False).features

        # Split VGG16 into blocks (until each pooling layer)
        self.conv0 = nn.Conv2d(5,3, kernel_size=1)
        self.conv1 = features[:4]   # until pool1
        self.conv2 = features[4:9]  # until pool2
        self.conv3 = features[9:16] # until pool3
        self.conv4 = features[16:23] # until pool4
        self.conv5 = features[23:30] # until pool5 (exclude pool5)

        # Side output layers
        self.side1 = nn.Conv2d(64, 1, kernel_size=1)
        self.side2 = nn.Conv2d(256, 1, kernel_size=1)
        self.side3 = nn.Conv2d(256, 1, kernel_size=1)
        self.side4 = nn.Conv2d(512, 1, kernel_size=1)
        self.side5 = nn.Conv2d(512, 1, kernel_size=1)

        # Fusion layer
        self.fusion = nn.Conv2d(5, 1, kernel_size=1)

        # Initialize weights for side outputs and fusion
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        h, w = x.size(2), x.size(3)

        # VGG blocks forward pass
        x=self.conv0(x)
        conv1 = self.conv1(x)
        conv2 = self.conv2(conv1)
        conv3 = self.conv3(conv2)
        conv4 = self.conv4(conv3)
        conv5 = self.conv5(conv4)

        # Side outputs
        side1 = self.side1(conv1)
        side2 = self.side2(conv2)
        side3 = self.side3(conv3)
        side4 = self.side4(conv4)
        side5 = self.side5(conv5)

        # Upsampling
        side1 = F.interpolate(side1, size=(h, w), mode='bilinear', align_corners=True)
        side2 = F.interpolate(side2, size=(h, w), mode='bilinear', align_corners=True)
        side3 = F.interpolate(side3, size=(h, w), mode='bilinear', align_corners=True)
        side4 = F.interpolate(side4, size=(h, w), mode='bilinear', align_corners=True)
        side5 = F.interpolate(side5, size=(h, w), mode='bilinear', align_corners=True)

        # Fusion of side outputs
        fused = torch.cat((side1, side2, side3, side4, side5), dim=1)
        fused = self.fusion(fused)

        # Apply sigmoid to all outputs
        # side1 = torch.sigmoid(side1)
        # side2 = torch.sigmoid(side2)
        # side3 = torch.sigmoid(side3)
        # side4 = torch.sigmoid(side4)
        # side5 = torch.sigmoid(side5)
        # fused = torch.sigmoid(fused)

        return side1, side2, side3, side4, side5, fused



# HED Loss
def hed_loss(side_outputs, fused, labels, beta=1.1):
    # Compute loss for each side output
    MSELoss=nn.MSELoss()
    loss1 = MSELoss(side_outputs[0], labels)
    loss2 =  MSELoss(side_outputs[1], labels)
    loss3 = MSELoss(side_outputs[2], labels)
    loss4 = MSELoss(side_outputs[3], labels)
    loss5 =  MSELoss(side_outputs[4], labels)

    # Compute loss for fused output
    loss_fused =MSELoss(fused, labels)

    # Total loss
    loss_total = loss1 + loss2 + loss3 + loss4 + loss5 + loss_fused

    return loss_total

In [6]:
# Prepare dataset and dataloader
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
lossesh=nn.MSELoss()
# Training loop
model = HED().to(device)
print('j')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
print('j')
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    val_loss=0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        # print('i')
        # Forward pass
        side1, side2, side3, side4,side5, fused = model(inputs)

        # Compute loss

        loss = hed_loss([side1, side2, side3, side4,side5], fused, labels)
        loss2 = lossesh(fused, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss2.item()
    # for inputs, labels in train_loader_2:
    #     inputs, labels = inputs.to(device), labels.to(device)
    #     # print('i')
    #     # Forward pass
    #     side1, side2, side3, side4, fused = model(inputs)

    #     # Compute loss

    #     loss = hed_loss([side1, side2, side3, side4], fused, labels)
    #     loss2 = lossesh(fused, labels)

    #     # Backward and optimize
    #     optimizer.zero_grad()
    #     loss.backward()
    #     optimizer.step()

    #     running_loss += loss2.item()
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        side1, side2, side3, side4,side5,  fused = model(inputs)

        # Compute loss

        loss = lossesh(fused, labels)



        val_loss += loss.item()
    print('val',(val_loss/len(val_loader))**0.5)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {((running_loss/(len(train_loader)))**0.5):.4f}")

In [7]:
# import os

#   # Replace with your folder path

# # Iterate through the folder
# for image_name in os.listdir('/kaggle/input/ee-5179-modern-computer-vision-course-competition/competition-data/testing-images'):
#     if image_name.endswith('.jpg') or image_name.endswith('.png'):  # Check for image files
#         image_path = os.path.join('/kaggle/input/ee-5179-modern-computer-vision-course-competition/competition-data/testing-images', image_name)

#         # Load and preprocess the imag`e
#         image = Image.open(image_path)
#         input_tensor = transform(image).unsqueeze(0)

#         # Perform inference
#         with torch.no_grad():
#             output = model(input_tensor.to(device))
#             output_image=(output[5][:][:][0][0]*255).cpu().numpy()
#             output_pil = Image.fromarray(output_image).convert('L')
#             output_pil.save('test_fold/'+image_name.split('.')[0]+'.jpg')
#              # Get the predicted class

#         # print(f"Image: {image_name}")


In [8]:
# ssim_module = SSIM(data_range=1, size_average=True, channel=3)


In [9]:
# pip install SSIM

In [10]:
# from pytorch_msssim import ssim, ms_ssim

In [11]:
pip install cv2

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
# import torch
# import torch.nn.functional as F

# def depth_smoothness_loss(depth, image):
#     """
#     Computes the depth smoothness loss.

#     Args:
#         depth (torch.Tensor): Predicted depth map of shape (B, 1, H, W).
#         image (torch.Tensor): Input image of shape (B, 3, H, W).

#     Returns:
#         torch.Tensor: Smoothness loss.
#     """
#     # Compute gradients of the depth map
#     depth_dx = torch.abs(depth[:, :, :, :-1] - depth[:, :, :, 1:])
#     depth_dy = torch.abs(depth[:, :, :-1, :] - depth[:, :, 1:, :])

#     # Compute gradients of the image
#     image_dx = torch.mean(torch.abs(image[:, :, :, :-1] - image[:, :, :, 1:]), dim=1, keepdim=True)
#     image_dy = torch.mean(torch.abs(image[:, :, :-1, :] - image[:, :, 1:, :]), dim=1, keepdim=True)

#     # Weight depth gradients by image gradients
#     weights_x = torch.exp(-image_dx)
#     weights_y = torch.exp(-image_dy)

#     smoothness_x = depth_dx * weights_x
#     smoothness_y = depth_dy * weights_y

#     return torch.mean(smoothness_x) + torch.mean(smoothness_y)


In [13]:
train_dataset[0][0].shape

NameError: name 'train_dataset' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.nn import Conv2d, BatchNorm2d,ReLU,MaxPool2d

class HED(nn.Module):
    def __init__(self):
        super(HED, self).__init__()
        self.conv0=Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding='same', bias=False)
        self.bn0=BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu0=ReLU(inplace=True)
        self.layer20=nn.Sequential(


       Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False,padding='same'),
        BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
      )
        self.layer30=nn.Sequential(
    Conv2d(256, 1024, kernel_size=(1, 1),stride=(1, 1), bias=False,padding='same'),
        BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
      )
        self.layer40=nn.Sequential(
    Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False,padding='same'),
        BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
      )
        self.layer11=nn.Sequential(

             Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer12=nn.Sequential(

             Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer13=nn.Sequential(

             Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )

        self.layer21=nn.Sequential(

             Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer22=nn.Sequential(

             Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer23=nn.Sequential(

             Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer24=nn.Sequential(

             Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer31=nn.Sequential(

             Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer32=nn.Sequential(

             Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )

        self.layer33=nn.Sequential(

             Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer34=nn.Sequential(

             Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer35=nn.Sequential(

             Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer36=nn.Sequential(

             Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer41=nn.Sequential(

             Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer42=nn.Sequential(

             Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )
        self.layer43=nn.Sequential(

             Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
               BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              ReLU(inplace=True),
              Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
             BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        )


        # self.conv5 = features[23:30] # until pool5 (exclude pool5)
        # self.linear=nn.Linear(1000,1024)
        # # Side output layers
        # self.side1 = nn.Conv2d(64, 1, kernel_size=1)
        # self.side2 = nn.Conv2d(128, 1, kernel_size=1)
        # self.side3 = nn.Conv2d(256, 1, kernel_size=1)
        # self.side4 = nn.Conv2d(512, 1, kernel_size=1)

        # self.side5 = nn.Conv2d(512, 1, kernel_size=1)

        # Fusion layer
         # Upscale x2

        self.decoder1= nn.Sequential(
            nn.ConvTranspose2d(in_channels= 1024,out_channels=1024, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels= 1024,out_channels=1024, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True))
        self.decoder2= nn.Sequential(
            nn.ConvTranspose2d(in_channels= 1024,out_channels=512, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels= 512,out_channels=256, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True))
        self.decoder3= nn.Sequential(
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True))
        self.decoder4= nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
        )
        self.decoder5= nn.Sequential(
            nn.ConvTranspose2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True))
        self.decoder6=nn.Sequential(
            nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=16, out_channels=8, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True))
        self.decoder7=nn.Sequential(
            nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True)
        )
        self.decoder8=nn.Sequential(
            nn.ConvTranspose2d(in_channels=8, out_channels=4, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True)
        )
        self.decoder9=nn.Sequential(
            nn.ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=2, out_channels=1, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1),# Upscale x2
            nn.ReLU(inplace=True))

        self.decoder10= nn.Sequential(
            nn.ConvTranspose2d(in_channels= 1024,out_channels=32, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels= 32,out_channels=1, kernel_size=3, stride=1, padding=1),  # Upscale x2
            nn.ReLU(inplace=True))

        # Initialize weights for side outputs and fusion
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        h, w = x.size(2), x.size(3)

        # VGG blocks forward pass



        x=self.conv0(x)
        x=self.bn0(x)
        x=self.relu0(x)
        conv1=self.layer11(x)
        x=x+conv1
        conv1=self.layer12(x)
        x=x+conv1
        conv1=self.layer13(x)
        x=x+conv1
        conv1=x
        conv2=self.layer20(x)
        x=conv2
        conv2=self.layer21(conv2)
        x=x.view(conv2.shape)+conv2
        conv2=self.layer22(x)
        x=x+conv2
        conv2=self.layer23(x)
        x=x+conv2
        conv2=self.layer24(x)
        x=x+conv2
        conv2=x
        conv3=self.layer30(x)
        x=conv3
        conv3=self.layer31(conv3)
        x=x.view(conv3.shape)+conv3
        conv3=self.layer32(x)
        x=x+conv3
        conv3=self.layer33(x)
        x=x+conv3
        conv3=self.layer34(x)
        x=x+conv3
        conv3=self.layer35(x)
        x=x+conv3
        conv3=self.layer36(x)
        x=x+conv3
        conv3=x
        conv4=self.layer40(x)
        x=conv4
        conv4=self.layer41(conv4)
        x=x.view(conv4.shape)+conv4
        conv4=self.layer42(x)
        x=x+conv4
        conv4=self.layer43(x)
        x=x+conv4
        # conv4=self.decoder1(conv4)
       
        # conv4=self.decoder2(conv4)
      
        # conv4=self.decoder3(conv4)
   
        # conv4=self.decoder4(conv4)
     
        # conv4=self.decoder5(conv4)
     
        # conv4=self.decoder6(conv4)

        # conv4=self.decoder7(conv4)
     
        # conv4=self.decoder8(conv4)
        # conv4=self.decoder9(conv4)
        conv4=decoder10(conv4)
        # conv5 = self.conv5(conv4)
        # for layer in self.decoder3:
        #  conv3 = layer(conv3)
        #  print(conv3.shape)

        # side5 = self.side5(conv5)




        # Apply sigmoid to all outputs
        # side1 = torch.sigmoid(side1)
        # side2 = torch.sigmoid(side2)
        # side3 = torch.sigmoid(side3)
        # side4 = torch.sigmoid(side4)
        # side5 = torch.sigmoid(side5)
        # fused = torch.sigmoid(fused)

        return conv4



# HED Loss
# def hed_loss(side_outputs, fused, labels, beta=1.1):
#     # Compute loss for each side output

#     loss1 = depth_smoothness_loss(side_outputs[0], labels)
#     loss2 =  depth_smoothness_loss(side_outputs[1], labels)
#     loss3 = depth_smoothness_loss(side_outputs[2], labels)
#     loss4 = depth_smoothness_loss(side_outputs[3], labels)
#     # loss5 =  depth_smoothness_loss(side_outputs[4], labels)

#     # Compute loss for fused output
#     loss_fused =depth_smoothness_loss(fused, labels)

#     # Total loss
#     loss_total = loss1 + loss2 + loss3 + loss4 +  loss_fused

#     return loss_total


In [ ]:
def lossesh(ytrue,ypred):
    mse=nn.MSELoss()
    mae=nn.L1Loss()
    l1var=torch.mean(torch.abs(ytrue-torch.mean((ytrue))))
    l2var=l2var = torch.std(ytrue, unbiased=False)
    return torch.nan_to_num((2*(mse(ytrue,ypred))**0.5+4*mae(ytrue,ypred)-l2var),nan=0)

In [ ]:
# Training loop
model = HED().to(device)

In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable Parameters: {trainable_params}")


In [ ]:
nig=nn.MSELoss()

In [ ]:
# Prepare dataset and dataloader
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch_xla.distributed.parallel_loader as pl

lossesh1=nn.MSELoss()

print('j')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
print('j')
num_epochs = 50
for epoch in range(5):
    model.train()
    running_loss = 0.0
    val_loss=0
    model.train()
    train_sampler.set_epoch(epoch)  # Ensure proper data distribution across TPU cores
    para_loader = pl.ParallelLoader(train_loader, [device])
    i=0
    for batch in para_loader.per_device_loader(device):
        inputs, labels =batch
        inputs, labels = inputs.to(device), labels.to(device)

        # print(i)
        i+=1
        # Forward pass
        fused = model(inputs)

        # Compute loss


        loss = lossesh(fused, labels)
        loss2=lossesh1(fused, labels)
        # print(loss2**0.5)
        # # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        xm.optimizer_step(optimizer)

        running_loss += loss2.item()
    # for inputs, labels in train_loader_2:
    #     inputs, labels = inputs.to(device), labels.to(device)
    #     # print('i')
    #     # Forward pass
    #     side1, side2, side3, side4, fused = model(inputs)

    #     # Compute loss

    #     loss = hed_loss([side1, side2, side3, side4], fused, labels)
    #     loss2 = lossesh(fused, labels)

    #     # Backward and optimize
    #     optimizer.zero_grad()
    #     loss.backward()
    #     optimizer.step()

    #     running_loss += loss2.item()
    para_loader = pl.ParallelLoader(val_loader, [device])
    for batch in para_loader.per_device_loader(device):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        fused = model(inputs)

        # Compute loss

        loss =nig(fused, labels)



        val_loss += loss.item()
    print('val',(val_loss/len(val_loader))**0.5)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {((running_loss/(len(train_loader)))**0.5):.4f}")

In [ ]:
import os

# Create a folder named 'my_folder' in the working directory
os.makedirs("/kaggle/working/test_fold", exist_ok=True)

In [ ]:
import os

  # Replace with your folder path

# Iterate through the folder
for image_name in os.listdir('/kaggle/input/ee-5179-modern-computer-vision-course-competition/competition-data/testing-images'):
    if image_name.endswith('.jpg') or image_name.endswith('.png'):  # Check for image files
        image_path = os.path.join('/kaggle/input/ee-5179-modern-computer-vision-course-competition/competition-data/testing-images', image_name)

        # Load and preprocess the imag`e
        image = Image.open(image_path)
        input_tensor = transform(image).unsqueeze(0)

        # Perform inference
        with torch.no_grad():
            output = model(input_tensor.to(device))
            output_image=(output[:][:][0][0]*255).cpu().numpy()
            output_pil = Image.fromarray(output_image).convert('L')
            output_pil.save('test_fold/'+image_name.split('.')[0]+'.jpg')
             # Get the predicted class

        # print(f"Image: {image_name}")


In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

def images_to_csv_with_metadata(image_folder, output_csv):
    # Initialize an empty list to store image data and metadata
    data = []

    # Loop through all images in the folder
    for idx, filename in enumerate(sorted(os.listdir(image_folder))):
        # print(filename.endswith(".png"))
        # if filename.endswith(".png"):

            # print(filename)
            filepath = os.path.join(image_folder, filename)
            # Read the image
            image = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)
            image = cv2.resize(image, (256, 256))
            image = image / 255.
            image = (image - np.min(image)) / (np.max(image) - np.min(image) + 1e-6)
            image = np.uint8(image * 255.)
            # Flatten the image into a 1D array
            image_flat = image.flatten()
            # Add ID, ImageID (filename), and pixel values
            row = [idx, filename] + image_flat.tolist()
            data.append(row)

    # Create a DataFrame
    num_columns = len(data[0]) - 2 if data else 0
    column_names = ["id", "ImageID"] + [indx for indx in range(num_columns)]
    df = pd.DataFrame(data, columns=column_names)

    # Save to CSV
    df.to_csv(output_csv, index=False)

# Paths for prediction and ground truth images
predictions_folder = "test_fold"

# Output CSV paths
predictions_csv = "predictions_hed_resss_cub7.csv"

# Convert prediction images to CSV
images_to_csv_with_metadata(predictions_folder, predictions_csv)

In [ ]:
print('jig')